In [ ]:
import pandas as pd
import numpy as np

In [ ]:
submit = pd.read_csv('submission.csv')

In [ ]:
submit

# NER

In [ ]:
stories = submit['story'].tolist()
stories

In [ ]:
submit.head(5)

In [ ]:
stories[:5]

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
# pip install sentence_transformers
# pip install faiss-cpu
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import pickle
import json
import re
# Load a multilingual model (important since your data is in Thai)
model = SentenceTransformer('BAAI/bge-m3')

# Create embeddings for the modified stories
story_texts = df['story'].tolist()
story_embeddings = model.encode(story_texts, show_progress_bar=True)

# Save embeddings and data for later use
embeddings_file = 'story_embeddings.npy'
np.save(embeddings_file, story_embeddings)

# Create an index mapping for retrieval
indices = {i: {'id': df.iloc[i]['id'], 
               'story': df.iloc[i]['story'],
               'answers': df.iloc[i]['answers']} 
          for i in range(len(df))}

with open('story_index.pkl', 'wb') as f:
    pickle.dump(indices, f)

print(f"Created embeddings with shape: {story_embeddings.shape}")

# Create a FAISS index for fast similarity search
dimension = story_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # Using L2 distance
index.add(story_embeddings)

# Save the index
faiss.write_index(index, 'story_faiss_index.bin')

print("FAISS index created and saved")

def count_people_in_text(text):
    """Count how many unique people entities are in the text"""
    pattern = r'\{people_(\d+)\}'
    matches = re.findall(pattern, text)
    return len(set(matches))

def retrieve_similar_cases(query_text, top_k=5):
    # Encode the query
    query_embedding = model.encode([query_text])
    
    # Search the FAISS index
    distances, indices = index.search(query_embedding, top_k)
    
    # Get the results
    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            'distance': distances[0][i],
            'similarity': 1/(1+distances[0][i]),  # Convert distance to similarity
            'id': df.iloc[idx]['id'],
            'story': df.iloc[idx]['story'],
            'answers': df.iloc[idx]['answers']
        })
    
    return results



In [ ]:
def rag_nega(query_text, top_k=5):
    # Encode the query
    query_embedding = model.encode([query_text])
    
    # Search the FAISS index
    distances, indices = index.search(query_embedding, top_k)
    
    # Get the results
    results = []
    for i, idx in enumerate(indices[0]):
        results.append({
            'distance': distances[0][i],
            'similarity': 1/(1+distances[0][i]),  # Convert distance to similarity
            'id': df.iloc[idx]['id'],
            'story': df.iloc[idx]['story'],
            'answers': df.iloc[idx]['answers']
        })
    
    return results

rag_nega('ผู้หญิงคนหนึ่งนั่งอยู่บนเก้าอี้ไม้ในห้องนั่งเล่นที่มีผ้าม่านสีขาวและโต๊ะกาแฟไม้สักอยู่ข้างๆ เธอสวมเสื้อยืดสีฟ้าและกางเกงยีนส์สีดำ')

<h1> PROMPT CRIMINAL PROCESS<h1>

In [ ]:
# pip install openai
# pip install ollama

from ollama import Client
from tqdm import tqdm
from openai import OpenAI
import time
import json
import os

# Initialize OpenAI client
client = OpenAI(
    api_key="sk-mmCj6woGf4I7HWyEqtjhlDm3nfNHNtwc4SctOTEMxJudnk5E",
    base_url="https://api.opentyphoon.ai/v1"
)

# Setup checkpoint file
CHECKPOINT_FILE = "processing_checkpoint.json"
checkpoint_data = {
    "last_processed_idx": -1,
    "processed_results": []
}

# Load checkpoint if exists
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, 'r', encoding='utf-8') as f:
        checkpoint_data = json.load(f)
    print(f"Resuming from checkpoint: {checkpoint_data['last_processed_idx'] + 1}/{len(stories)}")
else:
    print("Starting new processing")

ans_list = checkpoint_data["processed_results"]
start_idx = checkpoint_data["last_processed_idx"] + 1

# Add tqdm wrapper to show progress
for idx in tqdm(range(start_idx, len(stories)), desc="Processing Stories"):
    story = stories[idx]
    
    # Get similar cases for few-shot learning
    similar_cases = retrieve_similar_cases(story, top_k=10)
    
    # Prepare examples from similar cases
    examples = ""
    for i, case in enumerate(similar_cases[:7]):  # Use top 7 similar cases
        examples += "ข้อความ: " + case['story'] + "\n"
        examples += "ผลลัพธ์: " + case['answers'] + "\n\n"

    prompt = """
จงวิเคราะห์ข้อความทางกฎหมาย ทำหน้าที่อ่านข้อความและจัดประเภทความผิดของแต่ละบุคคลตามหมวดหมู่ที่กำหนดไว้ด้านล่าง และ ตามบุคคลที่กำหนดไว้ด้านล่างเท่านั้น:
Class 1: ไม่มีความผิดใดๆ เลย (No fault whatsoever)
Class 2: ลักทรัพย์หรือชิงทรัพย์ (Theft or robbery)
Class 3: กระทำโดยประมาท, ละเลย หรือไม่ได้ตั้งใจ ซึ่งทำให้เกิดความเสียหายต่อชีวิตหรือทรัพย์สิน (Negligence or unintentional harm)
Class 4: บุกรุกเคหสถานหรือบุกรุกเวลากลางคืน (Trespassing on a dwelling or at night)
Class 5: กระทำโดยเจตนา ก่อให้เกิดความเสียหายต่อร่างกาย, จิตใจ หรือทรัพย์สิน (Intentional harm)
Class 6: ฉ้อโกง, ปลอมแปลงเอกสาร หรือสิ่งของเพื่อการค้า (Fraud, forgery, counterfeiting)
Class 7: ทำให้ผู้อื่นเสียชีวิตไม่ว่าในรูปแบบใด (Causing death)

โปรดแสดงผลลัพธ์ในรูปแบบ:
[{'ชื่อบุคคล': [เลขคลาสของความผิด]}, ...]
ถ้าบุคคลใดไม่มีความผิด ให้ใส่ [1]
หากบุคคลหนึ่งกระทำผิดหลายข้อ ให้ใส่เลขหลายข้อใน list
ให้ใช้เฉพาะข้อมูลจากข้อความเท่านั้น ห้ามคาดเดาเกินสิ่งที่ระบุ
แยกชื่อบุคคลแต่ละคนให้ชัดเจน และใส่ผลลัพธ์เรียงตามลำดับที่ปรากฏในข้อความ

ตัวอย่าง:

""" + examples
    
    try:
        response = client.chat.completions.create(
            model="typhoon-v2.1-12b-instruct",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": f"ข้อความ: {story}"}
            ],
            max_tokens=2000,
            temperature=0.0
        )
        content = response.choices[0].message.content
        ans_list.append(content)
        
        # Update checkpoint
        checkpoint_data["last_processed_idx"] = idx
        checkpoint_data["processed_results"] = ans_list
        
        # Save checkpoint every 5 items or at the end
        if (idx + 1) % 5 == 0 or idx == len(stories) - 1:
            with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
                json.dump(checkpoint_data, f, ensure_ascii=False, indent=2)
            print(f"Checkpoint saved at index {idx}")
            
        print(f"Processing story {idx+1}/{len(stories)}")
        print(story)
        print(content)
        print("-" * 50)  # Adding a separator for better readability
        
        time.sleep(0.5)  # Adding a delay to avoid hitting the rate limit
        
    except Exception as e:
        print(f"Error processing item {idx}: {e}")
        # Save checkpoint on error
        with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
            json.dump(checkpoint_data, f, ensure_ascii=False, indent=2)
        print(f"Checkpoint saved at index {checkpoint_data['last_processed_idx']}")
        raise  # Re-raise the exception to stop execution

# Final update to submission dataframe
submit['pred'] = ans_list

print("Processing complete and results saved to submission_with_predictions.csv")

In [ ]:
submit.head()

<h1>SUBMISSION COLUMN<h1>

In [ ]:
new = submit
original_submission = submit.drop(columns=['story', 'answers'], inplace=True)
original_submission = original_submission.rename(columns={'pred': 'answers'})

# Create the final submission dataframe with columns in the right order
final_submission = pd.DataFrame()
final_submission['id'] = new['id']
final_submission['story'] = original_submission['story']  # Add the story column from original file
final_submission['answers'] = new['pred']  # Use the predictions as answers

# Display the first few rows to verify
final_submission.head()
def advanced_clean_json_string(text):
    """
    Process the answers column:
    1. Remove code block formatting if present
    2. Don't strip valid JSON formatting (quotes, brackets, etc.)
    """
    if isinstance(text, str):
        # Remove markdown code blocks if present
        if '```json' in text:
            text = text.replace('```json\n', '').replace('\n```', '')
        if '```' in text:
            text = text.replace('```', '')
        
        # Clean extra whitespace but preserve valid JSON structure
        text = text.strip()
        
        return text
    return text

# Apply the cleaning to create a properly formatted file
final_submission['answers'] = final_submission['answers'].apply(advanced_clean_json_string)

# Check the results
final_submission[['id', 'answers']].head(3)

In [ ]:
# Add a new cell to fix the CSV export
import csv

final_submission.to_csv('rag.csv', 
                       index=False, 
                       quoting=csv.QUOTE_NONNUMERIC,  # Quote all non-numeric fields
                       escapechar='\\',  # Use backslash to escape special characters
                       doublequote=True)  # Use double quotes to escape quotes

# Verify the file can be read back
try:
    test_read = pd.read_csv('rag.csv')
    print("File successfully created and can be read back!")
    print(test_read.head(3))
except Exception as e:
    print(f"Error: {e}")